In [2]:
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_validate, KFold
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score
import numpy as np

# Number of random trials
NUM_TRIALS = 10

# Load the dataset
iris = load_iris()
X_iris = iris.data
y_iris = iris.target

# Set up possible values of parameters to optimize over
p_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],'C': [1, 10, 100, 1000]},
          {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

# We will use a Support Vector Classifier with "rbf" kernel
svm = SVC()

# Arrays to store scores
train_scores_accuracy_avg = np.zeros(NUM_TRIALS)
train_scores_accuracy_std = np.zeros(NUM_TRIALS)
train_scores_fmacro_avg = np.zeros(NUM_TRIALS)
train_scores_fmacro_std = np.zeros(NUM_TRIALS)
train_scores_fmicro_avg = np.zeros(NUM_TRIALS)
train_scores_fmicro_std = np.zeros(NUM_TRIALS)
test_scores_accuracy_avg = np.zeros(NUM_TRIALS)
test_scores_accuracy_std = np.zeros(NUM_TRIALS)
test_scores_fmacro_avg = np.zeros(NUM_TRIALS)
test_scores_fmacro_std = np.zeros(NUM_TRIALS)
test_scores_fmicro_avg = np.zeros(NUM_TRIALS)
test_scores_fmicro_std = np.zeros(NUM_TRIALS)
best_params = {"C":[],"kernel":[],"gamma":[]}

# Loop for each trial
for i in range(NUM_TRIALS):

    # Choose cross-validation techniques for the inner and outer loops,
    # independently of the dataset.
    # E.g "GroupKFold", "LeaveOneOut", "LeaveOneGroupOut", etc.
    inner_cv = KFold(n_splits=10, shuffle=True, random_state=i)
    outer_cv = KFold(n_splits=10, shuffle=True, random_state=i)

    # Non_nested parameter search and scoring
    clf = GridSearchCV(estimator=svm, param_grid=p_grid, cv=inner_cv)
    clf.fit(X_iris, y_iris)
    best_params["C"].append(clf.best_params_["C"])
    best_params["kernel"].append(clf.best_params_["kernel"])
    if "gamma" in clf.best_params_:
        best_params["gamma"].append(clf.best_params_["gamma"])
    else:
        best_params["gamma"].append(0)

    # Nested CV with parameter optimization
    scoring = {'acc': 'accuracy',
               'fmacro': 'f1_macro',
               'fmicro': 'f1_micro'}

    nested_score = cross_validate(clf, X=X_iris, y=y_iris, scoring=scoring, cv=outer_cv, return_train_score=True)
    train_scores_accuracy_avg[i] = nested_score["train_acc"].mean()
    train_scores_accuracy_std[i] = nested_score["train_acc"].std()
    train_scores_fmacro_avg[i] = nested_score["train_fmacro"].mean()
    train_scores_fmacro_std[i] = nested_score["train_fmacro"].std()
    train_scores_fmicro_avg[i] = nested_score["train_fmicro"].mean()
    train_scores_fmicro_std[i] = nested_score["train_fmicro"].std()
    test_scores_accuracy_avg[i] = nested_score["test_acc"].mean()
    test_scores_accuracy_std[i] = nested_score["test_acc"].std()
    test_scores_fmacro_avg[i] = nested_score["test_fmacro"].mean()
    test_scores_fmacro_std[i] = nested_score["test_fmacro"].std()
    test_scores_fmicro_avg[i] = nested_score["test_fmicro"].mean()
    test_scores_fmicro_std[i] = nested_score["test_fmicro"].std()

In [3]:
p_grid

[{'kernel': ['rbf'], 'gamma': [0.001, 0.0001], 'C': [1, 10, 100, 1000]},
 {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

In [4]:
len(X_iris)

150

In [5]:
nested_score.keys()

dict_keys(['fit_time', 'score_time', 'test_acc', 'train_acc', 'test_fmacro', 'train_fmacro', 'test_fmicro', 'train_fmicro'])

In [6]:
best_params

{'C': [1, 1000, 1000, 1000, 1, 1, 1000, 1000, 10, 100],
 'kernel': ['linear',
  'rbf',
  'linear',
  'linear',
  'linear',
  'linear',
  'rbf',
  'rbf',
  'linear',
  'rbf'],
 'gamma': [0, 0.001, 0, 0, 0, 0, 0.001, 0.001, 0, 0.001]}

In [7]:
nested_score["test_acc"]

array([1.        , 1.        , 1.        , 0.93333333, 1.        ,
       0.93333333, 0.93333333, 1.        , 1.        , 1.        ])

In [8]:
test_scores_accuracy_avg

array([0.97333333, 0.95333333, 0.96      , 0.97333333, 0.98666667,
       0.98      , 0.96666667, 0.96      , 0.97333333, 0.98      ])

In [9]:
max(test_scores_accuracy_avg)

0.9866666666666667